<h1><center>Fetch Rewards Coding Exercise - Data Analyst</center></h1>
<center>Mengkai Qi</center>

In [293]:
#!pip3 install pandasql
from pandas.io.json import json_normalize
import pandas as pd
from pandasql import sqldf
import json
from datetime import datetime
import ast

## 1. Convert the json file into the structured table
### 1.1 Users Table

<h5>Extract, transform, and reorder the certain columns based on the question description<h5>
_id: user Id
<br>
state: state abbreviation
<br>
createdDate: when the user created their account
<br>
lastLogin: last time the user was recorded logging in to the app
<br>
role: constant value set to 'CONSUMER'
<br>
active: indicates if the user is active; only Fetch will de-activate an account with this flag


In [57]:
# load the data
users = open("/Users/mengkaiqi/Desktop/test/fetch_rewards/users.json", "r")
# count the length of the json file
len_user = sum(1 for line in open("/Users/mengkaiqi/Desktop/test/fetch_rewards/users.json", "r"))
# new data frame for saving structured data
df_users = pd.DataFrame()

# transform
for i in range(len_user):
        record = users.readline() # read lines
        info_user = json.loads(record) # load lines
        flatten_user = pd.json_normalize(info_user) # transform the data
        df_users = df_users.append(flatten_user) # add new lines
df_users.columns # get old column name to rename

Index(['active', 'role', 'signUpSource', 'state', '_id.$oid',
       'createdDate.$date', 'lastLogin.$date'],
      dtype='object')

In [62]:
# rename the col to match the question description
df_users.rename(columns={'_id.$oid': '_id',
                         'createdDate.$date':'createdDate',
                         'lastLogin.$date': 'lastLogin'}, inplace=True)

# reorder the col to match the question description
users_reorder_colnames = ['_id', 'state', 'createdDate',
                'lastLogin', 'role', 'active', 'signUpSource']
df_users = df_users.reindex(columns=users_reorder_colnames)

# transform the date col from seconds to datestamp
df_users['createdDate'] = pd.to_datetime(df_users['createdDate'],unit='ms')
df_users['lastLogin'] = pd.to_datetime(df_users['lastLogin'],unit='ms')

# overview new data
df_users.head()

,_id,state,createdDate,lastLogin,role,active,signUpSource
0,5ff1e194b6a9d73a3a9f1052,WI,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,True,Email
0,5ff1e194b6a9d73a3a9f1052,WI,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,True,Email
0,5ff1e194b6a9d73a3a9f1052,WI,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,True,Email
0,5ff1e1eacfcf6c399c274ae6,WI,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936,consumer,True,Email
0,5ff1e194b6a9d73a3a9f1052,WI,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,True,Email


### 1.2 Brands Table

<h5>Extract, transform, and reorder the certain columns based on the question description<h5>

_id: brand uuid
<br>
barcode: the barcode on the item
<br>
brandCode: String that corresponds with the brand column in a partner product file
<br>
category: The category name for which the brand sells products in
<br>
categoryCode: The category code that references a BrandCategory
<br>
cpg: reference to CPG collection
<br>
topBrand: Boolean indicator for whether the brand should be featured as a 'top brand'
<br>
name: Brand name

In [75]:
# load the data
brands = open("/Users/mengkaiqi/Desktop/test/fetch_rewards/brands.json", "r")
# count the length of the json file
len_brands = sum(1 for line in open("/Users/mengkaiqi/Desktop/test/fetch_rewards/brands.json", "r"))
# new data frame for saving structured data
df_brands = pd.DataFrame()

# transform
for m in range(len_brands):
        record = brands.readline() # read lines
        info_brands = json.loads(record) # load lines
        flatten_brands = pd.json_normalize(info_brands) # transform the data
        df_brands = df_brands.append(flatten_brands) # add new lines
df_brands.columns # get old column name

Index(['barcode', 'category', 'categoryCode', 'name', 'topBrand', '_id.$oid',
       'cpg.$id.$oid', 'cpg.$ref', 'brandCode'],
      dtype='object')

In [78]:
# rename the col to match the question description
df_brands.rename(columns={'_id.$oid': '_id',
                         'cpg.$id.$oid':'cpg_id',
                         'cpg.$ref': 'cpg_ref'}, inplace=True)

# reorder the col to match the question description
brands_reorder_colnames = ['_id', 'barcode', 'brandCode','category', 'categoryCode', 
                    'cpg_id', 'cpg_ref', 'topBrand', 'name']
df_brands = df_brands.reindex(columns=brands_reorder_colnames)

# overview new data
df_brands.head()

,_id,barcode,brandCode,category,categoryCode,cpg_id,cpg_ref,topBrand,name
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,601ac114be37ce2ead437550,Cogs,False,test brand @1612366101024
0,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,False,Starbucks
0,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,601ac142be37ce2ead437559,Cogs,False,test brand @1612366146176
0,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,601ac142be37ce2ead437559,Cogs,False,test brand @1612366146051
0,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,False,test brand @1612366146827


In [353]:
df_brands.to_csv('/Users/mengkaiqi/Desktop/test/fetch_rewards/brands.csv',index=False)

### 1.3 Receipts Table

<h5>Extract, transform, and reorder the certain columns based on the question description<h5>


_id: uuid for this receipt
<br>bonusPointsEarned: Number of bonus points that were awarded upon receipt completion
<br>bonusPointsEarnedReason: event that triggered bonus points
<br>createDate: The date that the event was created
<br>dateScanned: Date that the user scanned their receipt
<br>finishedDate: Date that the receipt finished processing
<br>modifyDate: The date the event was modified
<br>pointsAwardedDate: The date we awarded points for the transaction
<br>pointsEarned: The number of points earned for the receipt
<br>purchaseDate: the date of the purchase
<br>purchasedItemCount: Count of number of items on the receipt
<br>rewardsReceiptItemList: The items that were purchased on the receipt
<br>rewardsReceiptStatus: status of the receipt through receipt validation and processing
<br>totalSpent: The total amount on the receipt
<br>userId: string id back to the User collection for the user who scanned the receipt

In [358]:
# load the data
receipts = open("/Users/mengkaiqi/Desktop/test/fetch_rewards/receipts.json", "r")
# count the length of the json file
len_receipts = sum(1 for line in open("/Users/mengkaiqi/Desktop/test/fetch_rewards/receipts.json", "r"))
# new data frame for saving structured data
df_receipts = pd.DataFrame()

# transform
for m in range(len_receipts):
        record = receipts.readline() # read lines
        info_receipts = json.loads(record) # load lines
        flatten_receipts = pd.json_normalize(info_receipts) # transform the data
        df_receipts = df_receipts.append(flatten_receipts) # add new lines
df_receipts.columns # get old column name

Index(['bonusPointsEarned', 'bonusPointsEarnedReason', 'pointsEarned',
       'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus',
       'totalSpent', 'userId', '_id.$oid', 'createDate.$date',
       'dateScanned.$date', 'finishedDate.$date', 'modifyDate.$date',
       'pointsAwardedDate.$date', 'purchaseDate.$date'],
      dtype='object')

In [359]:
# rename the col to match the question description
df_receipts.rename(columns={'_id.$oid': '_id',
                          'createDate.$date':'createDate',
                          'dateScanned.$date': 'dateScanned',
                          'finishedDate.$date': 'finishedDate',
                          'modifyDate.$date': 'modifyDate',
                          'pointsAwardedDate.$date': 'pointsAwardedDate',
                          'purchaseDate.$date': 'purchaseDate'}, inplace=True)

# reorder the col to match the question description
receipts_reorder_colnames = ['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason','createDate', 'dateScanned', 
                             'finishedDate', 'modifyDate','pointsAwardedDate', 'pointsEarned', 'purchaseDate', 
                             'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent', 
                             'userId']
df_receipts = df_receipts.reindex(columns=receipts_reorder_colnames)

# transform the date col from seconds to datestamp
df_receipts['createDate'] = pd.to_datetime(df_receipts['createDate'],unit='ms')
df_receipts['dateScanned'] = pd.to_datetime(df_receipts['dateScanned'],unit='ms')
df_receipts['finishedDate'] = pd.to_datetime(df_receipts['finishedDate'],unit='ms')
df_receipts['modifyDate'] = pd.to_datetime(df_receipts['modifyDate'],unit='ms')
df_receipts['pointsAwardedDate'] = pd.to_datetime(df_receipts['pointsAwardedDate'],unit='ms')
df_receipts['purchaseDate'] = pd.to_datetime(df_receipts['purchaseDate'],unit='ms')

# overview new data
df_receipts.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
0,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
0,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b


In [360]:
df_receipts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1119 entries, 0 to 0
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   _id                      1119 non-null   object        
 1   bonusPointsEarned        544 non-null    float64       
 2   bonusPointsEarnedReason  544 non-null    object        
 3   createDate               1119 non-null   datetime64[ns]
 4   dateScanned              1119 non-null   datetime64[ns]
 5   finishedDate             568 non-null    datetime64[ns]
 6   modifyDate               1119 non-null   datetime64[ns]
 7   pointsAwardedDate        537 non-null    datetime64[ns]
 8   pointsEarned             609 non-null    object        
 9   purchaseDate             671 non-null    datetime64[ns]
 10  purchasedItemCount       635 non-null    float64       
 11  rewardsReceiptItemList   679 non-null    object        
 12  rewardsReceiptStatus     1119 non-nul

The column named rewardsReceiptItemList is special, because it still contain nested information in it
<br>
Try to explore and flatten again

In [361]:
# Select the first record
receipts_rewardsReceiptItemList[0:1]

0    [{'barcode': '4011', 'description': 'ITEM NOT ...
Name: rewardsReceiptItemList, dtype: object

In [365]:
# Extract the rewardsReceiptItemList columns in order to avoid impact for thr original data frame
receipt_items = df_receipts[['_id','rewardsReceiptItemList']]

# Transform each element of a list-like to a row
receipt_items = receipt_items.explode('rewardsReceiptItemList')
receipt_items.reset_index(inplace=True)

# Transform the json structure into data frame structure
df_receiptitem = pd.concat([receipt_items, json_normalize(receipt_items['rewardsReceiptItemList'].dropna())
                            .add_prefix('rewardsReceiptItemList_')], 
                           axis=1)

/Users/mengkaiqi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


In [368]:
# Drop the rewardsReceiptItemList in two tables in order to decrease the duplicate like "_x" "_y"
df_receiptitem.drop(['rewardsReceiptItemList'], axis=1,  inplace=True)
df_receipts.drop(['rewardsReceiptItemList'], axis=1,  inplace=True)
df_receiptitem.drop(['index'], axis=1,  inplace=True)

# Merge the two tables into one
df_receipts_final = pd.merge(df_receipts, df_receiptitem, 
                             left_index=True, right_index=True, how='outer')
df_receipts_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8499 entries, 0 to 7380
Data columns (total 49 columns):
 #   Column                                                     Non-Null Count  Dtype         
---  ------                                                     --------------  -----         
 0   _id_x                                                      1119 non-null   object        
 1   bonusPointsEarned                                          544 non-null    float64       
 2   bonusPointsEarnedReason                                    544 non-null    object        
 3   createDate                                                 1119 non-null   datetime64[ns]
 4   dateScanned                                                1119 non-null   datetime64[ns]
 5   finishedDate                                               568 non-null    datetime64[ns]
 6   modifyDate                                                 1119 non-null   datetime64[ns]
 7   pointsAwardedDate                

In [488]:
# export the data table to analysis it with MySQL
df_receiptitem.to_csv('/Users/mengkaiqi/Desktop/test/fetch_rewards/receipts_items.csv',index=False)
df_receipts.to_csv('/Users/mengkaiqi/Desktop/test/fetch_rewards/receipts.csv',index=False)
df_users.to_csv('/Users/mengkaiqi/Desktop/test/fetch_rewards/users.csv',index=False)

In [373]:
df_receiptitem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 35 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   _id                                                        7381 non-null   object 
 1   rewardsReceiptItemList_barcode                             3090 non-null   object 
 2   rewardsReceiptItemList_description                         6560 non-null   object 
 3   rewardsReceiptItemList_finalPrice                          6767 non-null   object 
 4   rewardsReceiptItemList_itemPrice                           6767 non-null   object 
 5   rewardsReceiptItemList_needsFetchReview                    813 non-null    object 
 6   rewardsReceiptItemList_partnerItemId                       6941 non-null   object 
 7   rewardsReceiptItemList_preventTargetGapPoints              358 non-null    object 
 8   rewardsR

## 2. Write a query that directly answers a predetermined question from a business stakeholder

For this part, some of the questions cannot be answered with the sqldf package, because this package cannot support datetime calculation.
<br>
So, I have another SQL file includes all SQL codes named "Part2_SQL.sql".
<br> The sqldf can relove the question 3 and 4 as shown below.

In [316]:
# specify globals() or locals() using the following helper function
mysql = lambda q: sqldf(q, globals())

3. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [399]:
query3 = """
    SELECT rewardsReceiptStatus, AVG(totalSpent) average_spend
    FROM df_receipts r
    WHERE rewardsReceiptStatus == 'FINISHED' OR rewardsReceiptStatus == 'REJECTED'
    GROUP BY rewardsReceiptStatus
    ORDER BY AVG(totalSpent) DESC
"""

mysql(query3)

,rewardsReceiptStatus,average_spend
0,FINISHED,80.854305
1,REJECTED,23.326056


4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [400]:
query4 = """
    SELECT r.rewardsReceiptStatus, COUNT(*) total_num
    FROM df_receipts r
    LEFT OUTER JOIN df_receiptitem ri ON r._id = ri._id
    WHERE rewardsReceiptStatus = 'FINISHED' OR rewardsReceiptStatus = 'REJECTED'
    GROUP BY rewardsReceiptStatus
    ORDER BY COUNT(r.rewardsReceiptStatus) DESC
"""

mysql(query4)

,rewardsReceiptStatus,total_num
0,FINISHED,5920
1,REJECTED,167


## 3. Evaluate Data Quality Issues in the Data Provided

In [480]:
# define a function to evaluate each table
def quality_overview(df):
    summary = pd.DataFrame(columns = ['Column Name', '% of Records','% Null Value', 'Unique Values', 'Most Common Value'])
    summary["Column Name"] = df.columns
    
    x = len(df.columns)
    
    for i in range(x):
        summary.iloc[i,1] = (df.iloc[:,i].notnull().sum())
        summary.iloc[i,2] = df.iloc[:,i].isnull().sum()
        summary.iloc[i,3] = len(df.iloc[:,i].unique())
        summary.iloc[i,4] = df.iloc[:,i].value_counts().head(1).index[0]
    
    return summary

In [481]:
quality_overview(df_users)

,Column Name,% of Records,% Null Value,Unique Values,Most Common Value
0,_id,495,0,212,54943462e4b07e684157a532
1,state,439,56,9,WI
2,createdDate,495,0,212,2014-12-19 14:21:22.381000
3,lastLogin,433,62,173,2021-03-05 16:52:23.204000
4,role,495,0,2,consumer
5,active,495,0,2,True
6,signUpSource,447,48,3,Email


#### For the user table, there are 2 data issues:
<br>1. user id has too many duplicte records, only 212 unique values
<br>2. state, lastLogin, signUpSource has null values, and that might cause some problems if we want to do some deeper analysis, like customer aqusistion analysis based on the signUpSource.

In [482]:
quality_overview(df_brands)

,Column Name,% of Records,% Null Value,Unique Values,Most Common Value
0,_id,1167,0,1167,5e5ff1f7ee7f2d0b35b2a18e
1,barcode,1167,0,1160,511111605058
2,brandCode,933,234,898,
3,category,1012,155,24,Baking
4,categoryCode,517,650,15,BAKING
5,cpg_id,1167,0,196,559c2234e4b06aca36af13c6
6,cpg_ref,1167,0,2,Cogs
7,topBrand,555,612,3,False
8,name,1167,0,1156,Huggies


#### For the brands table, there are 3 data findings/issues:
<br>1. brandCode, categoryCode has too many null values
<br>2. Baking is our main brand category
<br>3. We have more cooperation with brand Huggies than other brands

In [483]:
quality_overview(df_receipts)

,Column Name,% of Records,% Null Value,Unique Values,Most Common Value
0,_id,1119,0,1119,6011f4380a720f05350000b9
1,bonusPointsEarned,544,575,13,5.0
2,bonusPointsEarnedReason,544,575,10,All-receipts receipt bonus
3,createDate,1119,0,1107,2021-01-08 22:19:43
4,dateScanned,1119,0,1107,2021-01-08 22:19:43
5,finishedDate,568,551,554,2021-01-03 15:25:34
6,modifyDate,1119,0,1104,2021-01-03 15:25:36
7,pointsAwardedDate,537,582,524,2021-01-03 15:25:34
8,pointsEarned,609,510,121,5.0
9,purchaseDate,671,448,359,2020-08-17 00:00:00


#### For the brands table, there are 3 data findings/issues:
<br>1. Too many null values in multi columns
<br>2. Most of our customers spend 1 dollar on our paltform. Maybe they just wanted to try out Fetch Rewards, or because of one of our previous events.
<br>3. We have the most number of receipts on 2021-01-08

In [485]:
quality_overview(df_receiptitem)

,Column Name,% of Records,% Null Value,Unique Values,Most Common Value
0,_id,7381,0,1119,600f2fc80a720f0535000030
1,rewardsReceiptItemList_barcode,3090,4291,569,4011
2,rewardsReceiptItemList_description,6560,821,1890,ITEM NOT FOUND
3,rewardsReceiptItemList_finalPrice,6767,614,829,9.99
4,rewardsReceiptItemList_itemPrice,6767,614,829,9.99
5,rewardsReceiptItemList_needsFetchReview,813,6568,3,False
6,rewardsReceiptItemList_partnerItemId,6941,440,917,1
7,rewardsReceiptItemList_preventTargetGapPoints,358,7023,2,True
8,rewardsReceiptItemList_quantityPurchased,6767,614,14,1.0
9,rewardsReceiptItemList_userFlaggedBarcode,337,7044,7,034100573065


#### For the rewardsReceiptItemList table, there are 3 data findings/issues:
<br>1. The table structrue contains too many nested structures, especially column rewardsReceiptItemList in receipts.json. I hope we can split the receipts table into two parts: receipts and items.
<br>2. This table has too many null values, which has seriously affected normal data queries.
<br> For example, barcode has so many missing values
<br>3. BEN AND JERRYS ICE CREAM campaign has attracted the most purchases.
<br>4. Based on the finalPrice and itemPrice, we can see that our customer like the price tag $9.99

## 4. Communicate with Stakeholders

Hello,

Hope all is well with you! This is Jason from the data analytics team. I have received the users, receipts, and brands tables yesterday and have preprocessed, evaluated these datasets. I have found some data issues and inspirations that hope to share with you for deeper data analysis in the future.

1. The users table has too many duplicate records, especially the id column, which might cause serious problems in the next step of the analysis. I need to review the data collection step to resolve it.
2. The original JSON file includes many nested structures, like the rewardsReceiptItemList column in the receipts table. I need to flatten its structure or use a better normal form in the beginning stage. This will also be beneficial for future database expansion.
3. Some key columns of these three tables contain too many missing values. We need to clarify the cause of the problem, such as whether the value itself can be null, or whether it is a problem with data collection.
4. In the rewardsReceiptItemList of receipts table, the barcode column has a special value - 4011. However, this value does not appear in the barcode of the brand table. I need to know if this value is a test item, or what exactly this item represents.
5. The DateTime format should be transferred from UNIX format to normal DateTime (MM-DD-YY). The UNIX format is not a well-readable format for further analysis.

I am sure that you are extremely busy and occupied and I shall remain sensitive to your time. Please let me know if we can speak sometimes at your convenience to resolve the data problems and improve the data structures.

Thank you and have a great day!

Best,
Jason